# [Pythagorean Tuning](https://www.audiolabs-erlangen.de/resources/MIR/FMP/C1/C1E10_PythagoreanTuning.html)

> 最も古い調律法は、ギリシャの哲学者・数学者であるピタゴラス（紀元前6世紀）が導入したものである。ピタゴラス音律とは、すべての音程の周波数比が、倍音列に見られる3:2の比率に基づいている音楽の調律法です。この比率は「完全5度」とも呼ばれています。ここでは、根音の中心周波数𝜔（周波数比1に相当）から音階を構成する。その後、周波数値を3/2倍、必要に応じて2倍にすることで、全ての周波数値が𝜔～2⋅𝜔（周波数比1～2に対応）になるようにする。この手順を繰り返し、13個の周波数値（および13個の周波数比）を生成する。最後の(13番目の)度数比はピタゴラスコンマとも呼ばれ、完全五度だけで十二音音階を定義しようとした場合の矛盾の度合いを示している。


> 以下のコード例では、13の度数比を求めています。さらに、これらの度数比を、等音階から得られる度数比と比較する（差はセント単位で指定する）。

In [1]:
import numpy as np
import IPython.display as ipd
import sys
sys.path.append('FMP/')
import libfmp.c1

def compute_ratio(num3, num2):
    return (3 ** num3) / (2 ** num2)

num3 = 0
num2 = 0
note = 0
diff = 0
s = np.zeros((13, 6))
s[0] = [0, note, num3, num2, compute_ratio(num3, num2), diff]

for m in range(1, 13):
    note = (note + 7) % 12
    if note == 0:
        note = 12
    num3 = num3 + 1
    num2 = num2 + 1
    ratio = compute_ratio(num3, num2)
    if ratio > 2:
        num2 = num2 + 1
        ratio = compute_ratio(num3, num2)
    diff = (np.log2(ratio) - 1 / 12) * 1200
    diff = np.remainder(diff, 100)
    s[m] = [m, note, num3, num2, compute_ratio(num3, num2), diff]


In [10]:
s[0]

array([0., 0., 0., 0., 1., 0.])

In [9]:

r = compute_ratio(3,2)
print(r)

6.75


In [ ]:

for m in range(13):
    print('m = %2i, note = %2i, num3 = %2i, num2 = %2i, ratio = %6i:%6i = %7.4f, diff = %+6.2f'
          % (s[m, 0], s[m, 1], s[m, 2], s[m, 3], 3 ** s[m, 2], 2 ** s[m, 3], s[m, 4], s[m, 5]))

print('Pythagorean comma: %7.4f (%+6.2f cents)' % (s[12, 4], s[12, 5]))

dur = 4  # seconds
Fs = 4000  # sampling rate
freq = 440
x, t = libfmp.c1.generate_sinusoid(dur=dur, Fs=Fs, freq=freq)
freq_pyt_comma = freq * s[12, 4]
x_pyt_comma, t = libfmp.c1.generate_sinusoid(
    dur=dur, Fs=Fs, freq=freq_pyt_comma)

print()
print('Sinsoid of 440 Hz (A4):')
ipd.display(ipd.Audio(data=x, rate=Fs))
print('Sinsoid with %5.4f * 440 = %7.4f Hz:' % (s[12, 4], freq_pyt_comma))
ipd.display(ipd.Audio(data=x_pyt_comma, rate=Fs))


## Pythagorean Scale

> 最も古い調律法は、ギリシャの哲学者・数学者であるピタゴラス（紀元前6世紀）が導入したものである。前述したように、幾何学的な動機に基づくピタゴラス音律は、オクターブの周波数比2: 1と5分の1の周波数比3: 2にのみ基づいています。5分の1を加え、場合によってはオクターブを引くだけで、上記のような音階になります。5分音符とオクターブの足し算、引き算を許容すると、ピタゴラス音階になります。これにより、2の累乗または3の累乗のみを含む周波数比で表現できる音程が得られます。

> 以下のコードでは、C4から始まりC5で終わる半音階を考えます。13音すべてについて、等分音階の中心周波数とピタゴラス音程で得られる周波数を用いて、正弦波を生成します。


In [11]:
import pandas as pd
from collections import OrderedDict

# Computation of frequencies and differences
pyt_frac = ['1:1', '$2^8:3^5$', '$3^2:2^3$', '$2^5:3^3$', '$3^4:2^6$', '$2^2:3$', '$3^6:2^9$', 
                   '$3:2$', '$2^7:3^4$', '$3^3:2^4$', '$2^4:3^2$', '$3^5:2^7$', '$2:1$']
pyt_ratio = np.asarray([1, 256/243, 9/8, 32/27, 81/64, 4/3, 729/512, 3/2, 128/81, 27/16, 16/9, 243/128, 2])
p = 60
freq = libfmp.c1.f_pitch(p)
freq_pyt = pyt_ratio * freq
notes = np.asarray(range(p, p + 13))
freq_center = libfmp.c1.f_pitch(notes)
freq_deviation_cents =  libfmp.c1.difference_cents(freq_pyt, freq_center)

# Generation of sinusoids
dur = 4 #seconds
Fs = 4000 #sampling rate

sinusoid_freq_center = []
for freq in freq_center:
    x, t = libfmp.c1.generate_sinusoid(dur=dur, Fs=Fs, freq=freq)
    sinusoid_freq_center.append(x)

sinusoid_freq_pyt = []    
for freq in freq_pyt:
    x, t = libfmp.c1.generate_sinusoid(dur=dur, Fs=Fs, freq=freq)
    sinusoid_freq_pyt.append(x)    

# Generation of html table    
audio_tag_html_center = []
for i in range(len(freq_center)):
    audio_tag = ipd.Audio( sinusoid_freq_center[i], rate=Fs)
    audio_tag_html = audio_tag._repr_html_().replace('\n', '').strip()
    audio_tag_html = audio_tag_html.replace('<audio ', '<audio style="width: 100px; "')  
    audio_tag_html_center.append(audio_tag_html)
    
audio_tag_html_pyt = []
for i in range(len(freq_pyt)):
    audio_tag = ipd.Audio(sinusoid_freq_pyt[i], rate = Fs)
    audio_tag_html = audio_tag._repr_html_().replace('\n', '').strip()
    audio_tag_html = audio_tag_html.replace('<audio ', '<audio style="width: 100px; "')    
    audio_tag_html_pyt.append(audio_tag_html)

pd.set_option('display.max_colwidth', None)    
df = pd.DataFrame(OrderedDict([('Note', ['C4', 'C$^\sharp$4', 'D4', 'D$^\sharp$4', 'E4', 'F4',
                                         'F$^\sharp$4', 'G4', 'G$^\sharp$4', 'A4', 'A$^\sharp$4',
                                         'B4', 'C4']),
                               ('ET Freq. (Hz)', freq_center),
                               ('ET Sinusoid', audio_tag_html_center),  
                               ('Pyt. Ratio ', pyt_frac),
                               ('Pyt. Freq. (Hz)', freq_pyt),
                               ('Pyt. Sinusoid', audio_tag_html_pyt),                               
                               ('Difference (Cents)', freq_deviation_cents)]))

df.index = np.arange(1, len(df) + 1)
ipd.HTML(df.to_html(escape=False, float_format='%.2f'))

,Note,ET Freq. (Hz),ET Sinusoid,Pyt. Ratio,Pyt. Freq. (Hz),Pyt. Sinusoid,Difference (Cents)
1,C4,261.63,Your browser does not support the audio element.,1:1,261.63,Your browser does not support the audio element.,0.00
2,C$^\sharp$4,277.18,Your browser does not support the audio element.,$2^8:3^5$,275.62,Your browser does not support the audio element.,-9.78
3,D4,293.66,Your browser does not support the audio element.,$3^2:2^3$,294.33,Your browser does not support the audio element.,3.91
4,D$^\sharp$4,311.13,Your browser does not support the audio element.,$2^5:3^3$,310.07,Your browser does not support the audio element.,-5.87
5,E4,329.63,Your browser does not support the audio element.,$3^4:2^6$,331.12,Your browser does not support the audio element.,7.82
6,F4,349.23,Your browser does not support the audio element.,$2^2:3$,348.83,Your browser does not support the audio element.,-1.96
7,F$^\sharp$4,369.99,Your browser does not support the audio element.,$3^6:2^9$,372.51,Your browser does not support the audio element.,11.73
8,G4,392.00,Your browser does not support the audio element.,$3:2$,392.44,Your browser does not support the audio element.,1.96
9,G$^\sharp$4,415.30,Your browser does not support the audio element.,$2^7:3^4$,413.43,Your browser does not support the audio element.,-7.82
10,A4,440.00,Your browser does not support the audio element.,$3^3:2^4$,441.49,Your browser does not support the audio element.,5.87
